In [52]:
import os

In [53]:
%pwd

'd:\\ML-project\\Brain-Tumor'

In [42]:
os.chdir("../")

In [54]:
os.chdir("End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection/")

In [55]:
%pwd

'd:\\ML-project\\Brain-Tumor\\End-to-end-CNN-and-Hybrid-CNN-RF-Brain-Tumor-Detection'

In [63]:
# Entity for basemodel creation

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    base_model_source_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int

In [64]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [65]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            base_model_source_path=Path(config.base_model_source_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [66]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from keras import backend as K

In [67]:


def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [68]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.base_model_source_path)
        ft_input = self.model.input
        ft_layers = self.model.layers
        self.model = tf.keras.models.Model(inputs = ft_input, outputs = ft_layers[-4].output)
        self.save_model(path=self.config.base_model_path, model=self.model)

    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, learning_rate, freeze_till = None):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        feature_in = tf.keras.layers.Dropout(0.3)(model.output)
        feature_in = tf.keras.layers.BatchNormalization()(feature_in)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(feature_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["binary_accuracy", tf.keras.metrics.AUC()]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    


In [69]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-08-07 23:38:25,112: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 23:38:25,114: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 23:38:25,116: INFO: common: directory: artifacts created successfully]
[2023-08-07 23:38:25,117: INFO: common: directory: artifacts/prepare_base_model created successfully]
[2023-08-07 23:38:25,468: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 240, 240, 32)      2432      
                                                                 
 max_pooling2d_15 (MaxPoo

d:\ML-project\Brain-Tumor\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
